# TABLE DES MATIERES
* [INFORMATIONS ATTENDUES](#INFORMATIONS-ATTENDUES)
* [SOURCES](#SOURCES)
    * [ANNUAIRE](#ANNUAIRE)
    * [BDD TOPO](#BDD-TOPO)
    * [POTENTIEL GISEMENT SOLAIRE BRUT AU BATI](#POTENTIEL-GISEMENT-SOLAIRE-BRUT-AU-BATI)
* [EVALUATION](#EVALUATION)
    * [COMPLETUDE](#COMPLETUDE)
    * [PROXIMITE](#PROXIMITE)
        * [Similarité cosinus entre deux textes](#Similarité-cosinus-entre-deux-textes) 

Références: 
- https://outline.services.dataforgood.fr/doc/contraintes-a-respecter-pour-linstallation-de-panneaux-solaires-W2W2IaNwKn
- https://data-iau-idf.opendata.arcgis.com/datasets/iau-idf::le-potentiel-solaire-des-toitures/about
- https://outline.services.dataforgood.fr/doc/ressources-algorithme-9afGJyytC8
- https://geocatalogue.apur.org/catalogue/srv/fre/catalog.search#/metadata/urn:apur:potentiel_gisement_solaire_brut_au_bati

# INFORMATIONS ATTENDUES

## Contraintes à respecter
- Logistique
    - Pente du toit
    - Orientation au soleil
    - Latitude
    - Indice de rayonnement solaire local
    - Bâtiments classés
    - Bâtiments en zone non classée
    - Ombre (Bâtiments adjacents & végétations)
    - Eléments présents sur le toit
    - Structure porteuse (est-ce que le bâtiment peut supporter le poids?)
- Réseaux

# Potentiel Solaire - Attributs importants
- surf_util : Surface présentant une irradiation suffisante (> 900 kWh/m².an) et sans obstacles (comme des cheminées, velux, aérations, etc.). Cette surface est réellement utilisable pour l'installation de panneaux solaires.
- indic2 : Valeur numérique correspondant au gisement solaire. Les valeurs possibles sont :
1 : Non favorable
1 : Faible
2 : Intermédiaire
3 : Important
- gisement : Classification du gisement solaire en termes qualitatifs : "important", "intermédiaire", "faible", ou "Non favorable".
- moyenne2 : Potentiel solaire moyen au m² du bâtiment, exprimé en kWh/m².an. Cet indicateur représente la quantité d'énergie solaire reçue sur la surface utile.
- production : La production annuelle théorique en kWh/an, qui est une estimation de la quantité d'énergie solaire que la toiture pourrait produire sur une année.
- forme : Forme de la toiture estimée à partir d'un modèle numérique de surface. La forme de la toiture peut influencer l'ensoleillement et donc le potentiel solaire.
- eq_pano : La surface utile en équivalent panneaux solaires. Cet attribut indique le nombre de panneaux solaires susceptibles d'être installés sur la toiture.
- eq_surf : L'équivalent en superficie de l'équivalent panneau solaire, qui donne une idée de la surface disponible pour l'installation de panneaux solaires.

# Qualité du rattachement de l'annuaire avec les surfaces / bâtiments de la BD TOPO
"un des points importants à regarder en l'état c'est la qualité du rattachement de l'annuaire avec les surfaces / bâtiments de la BD TOPO. Pour le moment c'est une règle sur la proximité géographique qui est utilisée mais il y a d'autres métadonnées qui peuvent nous éclairer je pense. Par exemple :
le champs toponyme du layer zone d'activité de la BD TOPO (ex : "Ecole Elementaire Victor Hugo") avec le champs nom_etablissement de l'annuaire (ex: "Ecole élémentaire Victor Hugo")
le champs nature du layer zone d'activité de la BD TOPO (9 valeurs possibles) avec type_etablissement de l'annuaire (8 valeurs possibles)
En creusant, c'est clairement pas l'égalité à chercher mais peut etre calculer la proximité des valeurs (ou juste les comparer) pour voir si on s'est pas complètement loupé. Ou trouver une piste pour les établissements de l'annuaire sans surface d'activité.
Sachant que j'ai pas regardé tous les champs et il y a peut être d'autres informations à utiliser."

# SOURCES

## ANNUAIRE

In [1]:
import geopandas as gpd
from pathlib import Path
import matplotlib.pyplot as plt
import contextily as cx
import fiona
import pandas as pd

# Potentiel solaire package
from potentiel_solaire.constants import DATA_FOLDER

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [2]:
!extract-sample-data

2025-02-13 10:26:48,432 - INFO - root - /Users/hui-guan/Documents/Developpement/data4good/13_potentiel_solaire/algorithme/potentiel_solaire/extract_sample_data.py - main - GPKG file potentiel-solaire.gpkg already exists, skipping conversion.
2025-02-13 10:26:48,432 - INFO - root - /Users/hui-guan/Documents/Developpement/data4good/13_potentiel_solaire/algorithme/potentiel_solaire/extract_sample_data.py - main - GPKG file potentiel-gisement-solaire-brut-au-bati.gpkg already exists, skipping conversion.
2025-02-13 10:26:48,433 - INFO - root - /Users/hui-guan/Documents/Developpement/data4good/13_potentiel_solaire/algorithme/potentiel_solaire/extract_sample_data.py - main - Folder for BDTOPO_3-4_TOUSTHEMES_GPKG_LAMB93_D093_2024-12-15.7z already exists, skipping download and extraction.
2025-02-13 10:26:48,433 - INFO - root - /Users/hui-guan/Documents/Developpement/data4good/13_potentiel_solaire/algorithme/potentiel_solaire/extract_sample_data.py - main - Folder for PARCELLAIRE-EXPRESS_1-1__

In [3]:
annuaire = gpd.read_file(DATA_FOLDER / 'fr-en-annuaire-education.geojson')
annuaire = annuaire.to_crs(4326)
#annuaire.shape
annuaire[annuaire.code_commune=='93066'].sample(15) # Saint-Denis

,identifiant_de_l_etablissement,nom_etablissement,type_etablissement,statut_public_prive,adresse_1,adresse_2,adresse_3,code_postal,code_commune,nom_commune,...,code_type_contrat_prive,pial,etablissement_mere,type_rattachement_etablissement_mere,code_circonscription,code_zone_animation_pedagogique,libelle_zone_animation_pedagogique,code_bassin_formation,libelle_bassin_formation,geometry
41183,0930273S,Ecole élémentaire Honoré de Balzac,Ecole,Public,10 rue Jean Mermoz,None,93200 ST DENIS,93200,93066,Saint-Denis,...,99,0930587H,None,None,0932609F,None,None,None,None,POINT (2.35412 48.94141)
57569,0930505U,Ecole maternelle La Montjoie,Ecole,Public,8 rue Saint-Just,LA PLAINE SAINT DENIS,93200 ST DENIS,93200,93066,Saint-Denis,...,99,None,None,None,0931036W,None,None,None,None,POINT (2.36025 48.90835)
24233,0931490P,Collège La Courtille,Collège,Public,12 rue Jacques Vaché,None,None,93200,93066,Saint-Denis,...,99,None,None,None,None,None,None,24931,93 BASSIN 1 (DIST. 1 ET 2),POINT (2.38088 48.94016)
23035,0932665S,Ecole maternelle Le Corduan,Ecole,Public,12 mail des Maraichers,None,93200 ST DENIS,93200,93066,Saint-Denis,...,99,None,None,None,0931036W,None,None,None,None,POINT (2.36462 48.91399)
12092,0930208W,Ecole élémentaire Jules Guesde,Ecole,Public,8 rue du Corbillon,None,93200 ST DENIS,93200,93066,Saint-Denis,...,99,None,None,None,0931036W,None,None,None,None,POINT (2.35373 48.93804)
41483,0932673A,Ecole privée Al Andalus,Ecole,Privé,2 4 rue Jean-Baptiste Clément,None,93200 ST DENIS,93200,93066,Saint-Denis,...,10,None,None,None,0931036W,None,None,None,None,POINT (2.36888 48.93815)
12162,0930476M,Ecole maternelle Joncherolles,Ecole,Public,4 villa des Joncherolles,None,93200 ST DENIS,93200,93066,Saint-Denis,...,99,None,None,None,0932609F,None,None,None,None,POINT (2.35543 48.94863)
57787,0932409N,Ecole élémentaire Gutenberg,Ecole,Public,9 rue de la Montjoie,LA PLAINE SAINT DENIS,93200 ST DENIS,93200,93066,Saint-Denis,...,99,None,None,None,0931036W,None,None,None,None,POINT (2.35964 48.91094)
61821,0930865K,Collège Jean Lurçat,Collège,Public,22 rue d'Alembert,None,None,93200,93066,Saint-Denis,...,99,None,None,None,None,None,None,24931,93 BASSIN 1 (DIST. 1 ET 2),POINT (2.38006 48.93553)
61626,0930168C,Ecole élémentaire Louise Michel,Ecole,Public,35 rue Danielle Casanova,None,93200 ST DENIS,93200,93066,Saint-Denis,...,99,None,None,None,0932058G,None,None,None,None,POINT (2.36427 48.92859)


## BDD TOPO

In [5]:
GPKG = DATA_FOLDER / "BDTOPO_3-4_TOUSTHEMES_GPKG_LAMB93_D093_2024-12-15/BDTOPO/1_DONNEES_LIVRAISON_2024-12-00134/BDT_3-4_GPKG_LAMB93_D093-ED2024-12-15/BDT_3-4_GPKG_LAMB93_D093-ED2024-12-15.gpkg"
communes = gpd.read_file(GPKG, layer="commune")

# on filtre sur saint-denis
saint_denis = communes[communes.code_insee == "93066"].to_crs(4326)


In [6]:
# Exploration des layers
layers = fiona.listlayers(GPKG)
layers

['troncon_de_route',
 'route_numerotee_ou_nommee',
 'itineraire_autre',
 'troncon_de_voie_ferree',
 'equipement_de_transport',
 'piste_d_aerodrome',
 'aerodrome',
 'point_de_repere',
 'non_communication',
 'point_du_reseau',
 'voie_ferree_nommee',
 'toponymie_transport',
 'batiment',
 'cimetiere',
 'construction_lineaire',
 'construction_ponctuelle',
 'construction_surfacique',
 'reservoir',
 'ligne_orographique',
 'pylone',
 'terrain_de_sport',
 'toponymie_bati',
 'cours_d_eau',
 'troncon_hydrographique',
 'bassin_versant_topographique',
 'plan_d_eau',
 'surface_hydrographique',
 'noeud_hydrographique',
 'detail_hydrographique',
 'toponymie_hydrographie',
 'zone_d_habitation',
 'lieu_dit_non_habite',
 'detail_orographique',
 'toponymie_lieux_nommes',
 'canalisation',
 'ligne_electrique',
 'poste_de_transformation',
 'erp',
 'zone_d_activite_ou_d_interet',
 'toponymie_services_et_activites',
 'voie_nommee',
 'parc_ou_reserve',
 'foret_publique',
 'toponymie_zones_reglementees',
 'haie'

In [75]:
zone_d_activite=gpd.read_file(GPKG, layer="zone_d_activite_ou_d_interet")

In [42]:
zone_d_activite[zone_d_activite.categorie=='Science et enseignement']

,cleabs,categorie,nature,nature_detaillee,toponyme,statut_du_toponyme,importance,fictif,etat_de_l_objet,date_creation,date_modification,date_d_apparition,date_de_confirmation,sources,identifiants_sources,methode_d_acquisition_planimetrique,precision_planimetrique,identifiant_voie_ban,nom_commercial,geometry
27,SURFACTI0000000295936983,Science et enseignement,Structure d'accueil pour personnes handicapées,Etablissement et service d'aide par le travail,ESAT Ateliers de Chennevières,Collecté,5,True,En service,2012-02-17 10:09:31.224,2024-08-26 16:16:13.197,2017/01/03,2022-09-29,FINESS 2022,FINESS:940800170,Calculé,20.0,None,None,"MULTIPOLYGON (((666368.3 6855629.2, 666363.3 6..."
33,SURFACTI0000002206611076,Science et enseignement,Autre établissement d'enseignement,Centre de formation continue,Of-Cfa Afec,Collecté,6,True,En service,2020-04-22 17:54:14.640,2022-12-05 12:26:55.418,None,2022-10-19,MEN 2022,MEN:0756046P,Levé non GPS,5.0,None,None,"MULTIPOLYGON (((653850.8 6863186, 653845.8 686..."
36,SURFACTI0000000206971078,Science et enseignement,Lycée,None,Lycée Pour Handicapés votre École chez Vous,Collecté,6,True,En service,2008-10-03 16:13:30.756,2024-07-04 15:12:12.409,None,2024-07-04,MEN 2021,MEN:0754089M,Calculé,20.0,None,None,"MULTIPOLYGON (((654856.5 6862558.5, 654851.5 6..."
38,SURFACTI0000000356611011,Science et enseignement,Enseignement supérieur,None,Atelier Blanche Salant & Paul Weaver,Collecté,6,True,En service,2016-02-18 13:36:30.302,2024-07-09 10:20:43.092,None,2024-07-09,None,None,Calculé,20.0,None,None,"MULTIPOLYGON (((654664 6862999.9, 654659 68629..."
46,SURFACTI0000000106099761,Science et enseignement,Enseignement primaire,Ecole maternelle,École Ganénou Joëlle Msika-Annexe,Collecté,6,True,En service,2007-11-19 09:12:17.280,2024-07-09 17:28:15.581,None,2024-07-09,None,None,Calculé,20.0,None,None,"MULTIPOLYGON (((655351.5 6861455.2, 655346.5 6..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13339,SURFACTI0000000295401379,Science et enseignement,Enseignement supérieur,None,Institut de Formation en Soins Infirmiers Léon...,Collecté,5,False,En service,2012-02-13 13:27:16.754,2024-12-04 11:48:56.481,None,2024-04-09,MEN 2024,MEN:0951575W/MEN:0951987U,Orthophotographie,3.0,None,None,"MULTIPOLYGON (((655446.4 6876009.5, 655456.6 6..."
13340,SURFACTI0000000003753622,Science et enseignement,Enseignement primaire,Ecole maternelle,École Maternelle Lelong,Collecté,5,False,En service,2006-05-22 16:59:15.638,2024-12-04 11:48:56.481,None,2024-04-09,MEN 2024,MEN:0950613A,BDTopo,2.5,None,None,"MULTIPOLYGON (((654687.9 6877207.5, 654696.3 6..."
13341,SURFACTI0000000003753630,Science et enseignement,Collège,None,Collège Voltaire,Collecté,5,False,En service,2006-05-22 16:59:15.638,2024-12-04 11:48:56.481,None,2024-04-09,MEN 2024,MEN:0951196J,Orthophotographie,3.0,None,None,"MULTIPOLYGON (((654062.5 6877624.8, 654059 687..."
13343,SURFACTI0000002480826612,Science et enseignement,Enseignement primaire,Ecole maternelle,Annexe École Maternelle Marie Pape-Carpantier,Collecté,6,False,En service,2024-12-04 12:46:39.170,NaT,None,2024-12-04,None,MEN:0951263G,Orthophotographie,3.0,None,None,"MULTIPOLYGON (((655281.3 6878630.5, 655280.7 6..."


## POTENTIEL GISEMENT SOLAIRE BRUT AU BATI


Les attributs importants pour le projet sont en **gras** 

- N_SQ_EB: Identifiant informatique séquentiel
- **C_CAINSEE: Code INSEE de la commune/arrondissement**
- **C_ENS_MOY: Ensoleillement annuel moyen (KWh/m²/an)**
- M2_E_IN700: Surface d'ensoleillement inférieure à 700 KWh/m²/an
- M2_E_70_80: Surface d'ensoleillement comprise entre 700 et 800 KWh/m²/an
- M2_E_80_90: Surface d'ensoleillement comprise entre 800 et 900 KWh/m²/an
- **M2_E_90_10: Surface d'ensoleillement comprise entre 900 et 1000 KWh/m²/an**
- **M2_E_S1000: Surface d'ensoleillement supérieure à 1000 KWh/m²/an**
- **M2_E_TOT: Surface totale d'ensoleillement (m²)**
- SHAPE_LENGTH: Périmètre de l’emprise (m)
- SHAPE_AREA: Surface de l’emprise (m²)

In [37]:
potentiel_gisement_solaire = gpd.read_file(DATA_FOLDER / 'potentiel-gisement-solaire-brut-au-bati.geojson')
potentiel_gisement_solaire = potentiel_gisement_solaire.to_crs(4326)

In [38]:
potentiel_gisement_solaire[potentiel_gisement_solaire.c_cainsee==93066]

,geo_point_2d,geo_shape,objectid,n_sq_eb,n_sq_eb_or,c_cainsee,m2_e_in700,m2_e_70_80,m2_e_80_90,m2_e_90_10,m2_e_s1000,m2_e_tot,c_ens_moy,shape_leng,shape_area,geometry
33,"{ ""lon"": 2.3796635368542507, ""lat"": 48.9373916...",None,970114,930305028.0,930305028.0,93066.0,30,5,4,13,13,65,764.0,32.025768,63.961840,"POLYGON ((2.37974 48.93738, 2.37965 48.93734, ..."
207,"{ ""lon"": 2.3806389167648239, ""lat"": 48.9400496...",None,970618,930301319.0,930301319.0,93066.0,25,17,14,22,6,84,740.0,36.631041,83.173984,"POLYGON ((2.38072 48.94003, 2.38062 48.93999, ..."
362,"{ ""lon"": 2.3580500444470265, ""lat"": 48.9047674...",None,971039,930310138.0,930310138.0,93066.0,8,6,20,42,12,88,890.0,41.700756,88.229133,"POLYGON ((2.35813 48.9048, 2.35813 48.90474, 2..."
492,"{ ""lon"": 2.3601220243954284, ""lat"": 48.9390812...",None,971471,930302230.0,930302230.0,93066.0,127,5,0,0,0,132,297.0,57.607366,131.790633,"POLYGON ((2.36007 48.93914, 2.36011 48.93909, ..."
613,"{ ""lon"": 2.3716559323829625, ""lat"": 48.9418521...",None,971877,930299244.0,930299244.0,93066.0,47,4,3,7,30,91,718.0,44.288657,91.076052,"POLYGON ((2.37172 48.94183, 2.37172 48.9418, 2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075300,"{ ""lon"": 2.3755250187671768, ""lat"": 48.9408816...",None,889287,930301007.0,930301007.0,93066.0,2,1,0,0,0,3,690.0,8.664888,3.060450,"POLYGON ((2.37554 48.94088, 2.37551 48.94087, ..."
1075695,"{ ""lon"": 2.3761179299949657, ""lat"": 48.9396885...",None,900474,930301102.0,930301102.0,93066.0,8,6,22,29,4,69,869.0,33.067579,68.266061,"POLYGON ((2.37612 48.93965, 2.37607 48.93964, ..."
1075704,"{ ""lon"": 2.3496398245637113, ""lat"": 48.9358866...",None,887503,930303635.0,930303635.0,93066.0,34,14,20,6,1,75,724.0,51.245270,77.315867,"POLYGON ((2.34967 48.93579, 2.34962 48.93579, ..."
1075705,"{ ""lon"": 2.3793291458609791, ""lat"": 48.9346511...",None,887505,930305314.0,930305314.0,93066.0,13,14,28,39,16,110,864.0,54.153225,109.762669,"POLYGON ((2.37943 48.93461, 2.37946 48.9346, 2..."


## POTENTIEL SOLAIRE

In [76]:
potentiel = gpd.read_file(DATA_FOLDER / 'potentiel-solaire.geojson')

In [77]:
potentiel

,objectid,id,nature,surf_util,indic2,gisement,eq_pano,eq_surf,systeme,protection,mos2017,insee,moyenne2,forme,production,mos17,st_area(shape),st_length(shape),geometry
0,1,BATIMENT0000000356480536,Bâtiment industriel,382.50,3,important,plus de 50 panneaux,plus de 115 m2,thermique ou photovoltaïque,0,45,95078,1210.328054,plat,42184.047797,5,609.870,108.056166,"POLYGON ((2.03865 49.08767, 2.03867 49.08767, ..."
1,2,BATIMENT0000000356480545,Bâtiment industriel,83.25,2,intermédiaire,10 à 50 panneaux,entre 20 et 115 m2,thermique ou photovoltaïque,0,45,95078,1197.261665,Npans,12199.856914,5,159.180,51.796212,"POLYGON ((2.04011 49.08265, 2.04006 49.08257, ..."
2,3,BATIMENT0000000356475551,Bâtiment industriel,213.75,3,important,plus de 50 panneaux,plus de 115 m2,thermique ou photovoltaïque,1,45,95211,1204.673158,plat,23463.298631,5,713.220,115.314199,"POLYGON ((2.1148 49.06608, 2.11477 49.06605, 2..."
3,4,BATIMENT0000000317228796,Bâtiment industriel,240.75,3,important,plus de 50 panneaux,plus de 115 m2,thermique ou photovoltaïque,1,45,95211,1243.157793,toit2pentes,36633.125213,5,729.620,119.633673,"POLYGON ((2.11964 49.06853, 2.11947 49.06864, ..."
4,5,BATIMENT0000000003786796,Bâtiment industriel,132.75,3,important,plus de 50 panneaux,plus de 115 m2,thermique ou photovoltaïque,1,45,95370,1210.220040,plat,14639.039443,5,305.425,74.696097,"POLYGON ((1.97282 49.14438, 1.97277 49.14436, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,994,BATIMENT0000000003957857,Serre,126.00,3,important,plus de 50 panneaux,plus de 115 m2,thermique ou photovoltaïque,0,10,95018,1305.472070,plat,14988.281492,1,737.580,117.842566,"POLYGON ((2.2141 48.95716, 2.21353 48.95719, 2..."
996,995,BATIMENT0000000003957856,Serre,139.50,3,important,plus de 50 panneaux,plus de 115 m2,thermique ou photovoltaïque,0,10,95018,1179.596782,plat,14994.137799,1,374.490,98.708164,"POLYGON ((2.21411 48.95717, 2.21412 48.95725, ..."
997,996,BATIMENT0000000003957858,Serre,85.50,2,intermédiaire,10 à 50 panneaux,entre 20 et 115 m2,thermique ou photovoltaïque,0,10,95018,1236.205659,toit2pentes,12937.139463,1,322.970,89.279465,"POLYGON ((2.21413 48.95729, 2.21414 48.95737, ..."
998,997,BATIMENT0000000003953824,Serre,326.25,3,important,plus de 50 panneaux,plus de 115 m2,thermique ou photovoltaïque,0,10,95018,1198.183775,plat,35619.487445,1,959.600,144.546231,"POLYGON ((2.21968 48.96245, 2.21988 48.96238, ..."


# EVALUATION

## COMPLETUDE

### Valeurs manquantes

In [48]:
taux_noms_etablissements_manquants=annuaire.nom_etablissement.isnull().sum()/len(annuaire.nom_etablissement)
print(f"Taux de noms d'établissement manquants: {taux_noms_etablissements_manquants} %")

Taux de noms d'établissement manquants: 0.0 %


In [50]:
taux_toponymes_manquants=zone_d_activite.toponyme.isnull().sum()/len(zone_d_activite.toponyme)
print(f"Taux de toponymes*100 manquants: {taux_toponymes_manquants*100} %") 

Taux de toponymes*100 manquants: 8.667965238237938 %


### Couverture temporelle

In [58]:
annuaire[(annuaire.type_etablissement=='Ecole') & (annuaire.code_commune=='93066')].date_maj_ligne.unique()

<DatetimeArray>
['2025-02-02 00:00:00']
Length: 1, dtype: datetime64[ms]

In [63]:
zone_d_activite[zone_d_activite.categorie=='Science et enseignement'][["date_modification"]].sort_values(by='date_modification',ascending=False)

,date_modification
13344,2024-12-05 08:50:36.735
9400,2024-12-04 12:46:39.170
13341,2024-12-04 11:48:56.481
13339,2024-12-04 11:48:56.481
13340,2024-12-04 11:48:56.481
...,...
8539,NaT
8540,NaT
8623,NaT
8788,NaT


## PROXIMITE

Pour voir si les informations sont bien correlées, essayons de calculer la proximité des valeurs (ou juste les comparer) entre sources de données 

In [78]:
# recherche avec suppression préalable des accents (normalisation + encodage + desencodage + minuscule) 
s_annuaire_ecole_1=annuaire[annuaire.nom_etablissement.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()=="ecole elementaire daniel sorano"]
s_annuaire_ecole_1

,identifiant_de_l_etablissement,nom_etablissement,type_etablissement,statut_public_prive,adresse_1,adresse_2,adresse_3,code_postal,code_commune,nom_commune,code_departement,code_academie,code_region,ecole_maternelle,ecole_elementaire,voie_generale,voie_technologique,voie_professionnelle,telephone,fax,web,mail,restauration,hebergement,ulis,apprentissage,segpa,section_arts,section_cinema,section_theatre,section_sport,section_internationale,section_europeenne,lycee_agricole,lycee_militaire,lycee_des_metiers,post_bac,appartenance_education_prioritaire,greta,siren_siret,nombre_d_eleves,fiche_onisep,position,type_contrat_prive,libelle_departement,libelle_academie,libelle_region,coordx_origine,coordy_origine,epsg_origine,nom_circonscription,latitude,longitude,precision_localisation,date_ouverture,date_maj_ligne,etat,ministere_tutelle,multi_uai,rpi_concentre,rpi_disperse,code_nature,libelle_nature,code_type_contrat_prive,pial,etablissement_mere,type_rattachement_etablissement_mere,code_circonscription,code_zone_animation_pedagogique,libelle_zone_animation_pedagogique,code_bassin_formation,libelle_bassin_formation,geometry
41191,0930306C,Ecole élémentaire Daniel Sorano,Ecole,Public,3 boulevard Félix Faure,None,93200 ST DENIS,93200,93066,Saint-Denis,093,24,11,0.0,1.0,None,None,None,0183722220,None,None,ce.0930306C@ac-creteil.fr,1.0,0.0,0.0,None,None,None,None,None,None,None,None,None,None,None,None,REP,None,21930066200208,292.0,None,None,SANS OBJET,Seine-Saint-Denis,Créteil,Ile-de-France,652972.5,6871169.1,EPSG:2154,Circonscription d'inspection du 1er degré de S...,48.938774,2.35805,Numéro de rue,1965-05-01,2025-02-02,OUVERT,MINISTERE DE L'EDUCATION NATIONALE,0,0.0,None,151,ECOLE DE NIVEAU ELEMENTAIRE,99,None,None,None,0932609F,None,None,None,None,POINT (2.35805 48.93877)


In [80]:
s_bddtopo_ecole_1=zone_d_activite[zone_d_activite.toponyme.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.contains("ecole elementaire daniel sorano")==True]
s_bddtopo_ecole_1

,cleabs,categorie,nature,nature_detaillee,toponyme,statut_du_toponyme,importance,fictif,etat_de_l_objet,date_creation,date_modification,date_d_apparition,date_de_confirmation,sources,identifiants_sources,methode_d_acquisition_planimetrique,precision_planimetrique,identifiant_voie_ban,nom_commercial,geometry
9968,SURFACTI0000000002555607,Science et enseignement,Enseignement primaire,Groupe scolaire,École Élémentaire Daniel Sorano,Collecté,5,False,En service,2006-05-22 16:31:28.175,2024-04-17 14:57:25.903,None,2024-04-09,MEN 2024,MEN:0930306C,BDTopo,2.5,None,None,"MULTIPOLYGON (((653017.7 6871189.3, 653017.4 6..."


In [114]:
s_annuaire_ecole_1[['nom_etablissement','type_etablissement']]

,nom_etablissement,type_etablissement
41191,Ecole élémentaire Daniel Sorano,Ecole


In [85]:
s_bddtopo_ecole_1[['toponyme','nature']]

,toponyme,nature
9968,École Élémentaire Daniel Sorano,Enseignement primaire


### Similarité cosinus entre deux textes

Nous calculons la similarité cosinus entre deux attributs texte pour déterminer le taux de similarité  
Plus la valeur de la similarité cosinus est :  
- proche de 1, plus les textes sont similaires.  
- proche de 0, plus les textes sont différents.

In [104]:
import nltk
nltk.download("stopwords")

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from unidecode import unidecode

[nltk_data] Downloading package stopwords to /Users/hui-
[nltk_data]     guan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [127]:
def similarite_cosinus(X,Y):
    # Programme pour mesurer la similarité entre
    # deux phrases en utilisant la similarité cosinus (similarité=1 -> identique, similarité=0 -> pas du tout similaire)

    # Suppression des accents et passage en minuscules
    X = unidecode(X).lower()
    Y = unidecode(Y).lower()
    
    # Tokenisation
    X_list = word_tokenize(X)
    Y_list = word_tokenize(Y)
    
    # sw contient la liste des mots vides
    sw = set(stopwords.words('french'))
    l1 = []
    l2 = []
    
    # Supprimer les mots vides de la chaîne
    X_set = {w for w in X_list if w not in sw}
    Y_set = {w for w in Y_list if w not in sw}
    
    # Former un ensemble contenant les mots-clés des deux chaînes
    rvector = X_set.union(Y_set)
    for w in rvector:
        if w in X_set:
            l1.append(1)  # créer un premier vecteur
        else:
            l1.append(0)
        if w in Y_set:
            l2.append(1) # créer un deuxieme vecteur
        else:
            l2.append(0)
    c = 0
    
    # Formule du cosinus
    for i in range(len(rvector)):
        c += l1[i] * l2[i]
    cosine = c / float((sum(l1) * sum(l2)) ** 0.5)
    return cosine

In [124]:
X=s_annuaire_ecole_1['nom_etablissement'].values[0]
Y=s_bddtopo_ecole_1['toponyme'].values[0]

print(f"similarité ({X}|{Y}): {similarite_cosinus(X,Y)}")

unidecode: ecole elementaire daniel sorano, ecole elementaire daniel sorano
similarité (Ecole élémentaire Daniel Sorano|École Élémentaire Daniel Sorano): 1.0


Les attributs annuaire.nom_etablissement et bddtop.toponyme sont identiques

In [125]:
X=s_annuaire_ecole_1['type_etablissement'].values[0]
Y=s_bddtopo_ecole_1['nature'].values[0]

print(f"similarité ({X}|{Y}): {similarite_cosinus(X,Y)}")

unidecode: ecole, enseignement primaire
similarité (Ecole|Enseignement primaire): 0.0
